In [1]:
from datasets import load_dataset, load_from_disk, concatenate_datasets, get_dataset_config_names
# from promptsource import templates
import json
import pickle
import collections
import pandas as pd
from src.utils import concatenate_columns, count_gpt2_tokens
from src.wimbd_ import _load_dataset

CACHE_DIR = "/share/edc/home/antonis/datasets/huggingface"
import os
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR

# # Get a list of all supported datasets
# datasets = templates.get_dataset_names()
# print(datasets)

# datasets = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1/ds_c4_small")
# datasets = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1/c4_mixed_Commonsense")

[nltk_data] Downloading package punkt to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /local/home/antonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [4]:
import gzip

pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/wimbd/test_fixtures/c4-sample.00000-of-00001.json.gz"
# Open the file with gzip and read lines
with gzip.open(pth, 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            # Try to parse each line as a separate JSON object
            ds = json.loads(line)
            # Process the ds as needed
        except json.JSONDecodeError as e:
            print(f"Failed to decode line: {e}")
# ds = load_dataset("sciq", cache_dir=CACHE_DIR)
# ds = load_dataset("allenai/ai2_arc", 'ARC-Easy', cache_dir=CACHE_DIR)

In [16]:
ds_tulu = load_dataset("allenai/tulu-v2-sft-mixture", cache_dir=CACHE_DIR)

In [29]:
ds_tulu['train'][0]['messages'][0]

{'role': 'user',
 'content': 'Question: Gdańsk (, ; German: "" , ) is a Polish city on the Baltic coast. It is the capital of the Pomeranian Voivodeship, Poland\'s principal seaport and is also the centre of the country\'s fourth-largest metropolitan area. \n\nThe city lies on the southern edge of Gdańsk Bay (of the Baltic Sea), in a conurbation with the city of Gdynia, spa town of Sopot, and suburban communities, which together form a metropolitan area called the Tricity ("Trójmiasto"), with a population approaching 1.4 million. Gdańsk itself has a population of 460,427 (December 2012), making it the largest city in the Pomerania region of Northern Poland. \n\nGdańsk is the capital of Gdańsk Pomerania and the largest city of Kashubia. With its origins as a Polish stronghold erected in the 980s by Mieszko I of Poland, the city\'s history is complex, with periods of Polish rule, periods of Prussian or German rule, and periods of autonomy or self-rule as a "free city". Between the world 

In [30]:
ds_tulu['train'][0]['messages'][1]

{'role': 'assistant',
 'content': '1. Miguel\n2. Yes\n3. Eight\n4. unknown\n5. Swift\n6. "We Are Never Ever Getting Back Together"\n7. Female\n8. The rough times he has had of late.\n9. "Red"\n10. Yes'}

In [13]:
import gzip

pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/wimbd/test_fixtures/c4-sample.00000-of-00001.json.gz"
# Open the file with gzip and read lines
with gzip.open(pth, 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            # Try to parse each line as a separate JSON object
            ds = json.loads(line)
            # Process the ds as needed
        except json.JSONDecodeError as e:
            print(f"Failed to decode line: {e}")

ds.keys()

dict_keys(['text', 'url', 'date'])

In [19]:
ds_mmlu = _load_dataset("mmlu", CACHE_DIR=CACHE_DIR)

Loading dataset mmlu


In [20]:
ds_mmlu

{'abstract_algebra': DatasetDict({
     auxiliary_train: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 99842
     })
     test: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 100
     })
     validation: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 11
     })
     dev: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 5
     })
 }),
 'anatomy': DatasetDict({
     auxiliary_train: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 99842
     })
     test: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 135
     })
     validation: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 14
     })
     dev: Dataset({
         features: ['question', 'choices', 'answer'],
         num_rows: 5
     })
 }),
 'astronomy': DatasetDict({
     auxiliary_train: D

In [13]:
len(ds_['question'][0])

416

In [14]:
question_lengths

[181, 40, 64, 85, 166]

In [10]:
# get avg length of questions for different splits

for task in ds.keys():
    for split in ds[task].keys():
        ds_ = ds[task][split]
        question_lengths = [len(question) for question in ds_['question']]
        avg_length = sum(question_lengths) / len(question_lengths) if len(question_lengths) > 0 else 0
        print(task, split, avg_length)

144938185
abstract_algebra auxiliary_train 1451.6754972857113
12377
abstract_algebra test 123.77
1367
abstract_algebra validation 124.27272727272727
516
abstract_algebra dev 103.2
144938185
anatomy auxiliary_train 1451.6754972857113
11117
anatomy test 82.34814814814816
972
anatomy validation 69.42857142857143
261
anatomy dev 52.2
144938185
astronomy auxiliary_train 1451.6754972857113
13075
astronomy test 86.01973684210526
1330
astronomy validation 83.125
424
astronomy dev 84.8
144938185
business_ethics auxiliary_train 1451.6754972857113
15777
business_ethics test 157.77
1297
business_ethics validation 117.9090909090909
1087
business_ethics dev 217.4
144938185
clinical_knowledge auxiliary_train 1451.6754972857113
17581
clinical_knowledge test 66.3433962264151
1870
clinical_knowledge validation 64.48275862068965
413
clinical_knowledge dev 82.6
144938185
college_biology auxiliary_train 1451.6754972857113
19858
college_biology test 137.90277777777777
2057
college_biology validation 128.562

KeyboardInterrupt: 

In [ ]:
pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/7/all/es-en.pkl"
df = pd.DataFrame(pickle.load(open(pth, "rb"))).T.sort_values("value", ascending=False)

In [ ]:
df = df.reset_index().rename(columns={'index': 'ngram'})

In [ ]:
df

# choose only rows were ngram column has more 7 or more ngrams

df = df[df["ngram"].apply(lambda x: len(x.split()) >= 5)]
df = df[df["value"] > 0].reset_index(drop=True)
display(df)

In [ ]:
# europarl = load_dataset("wmt/europarl", cache_dir=CACHE_DIR)

In [ ]:
wmt14 = load_dataset("wmt14", 'cs-en', cache_dir=CACHE_DIR)

In [ ]:
wmt09 = load_from_disk("/share/edc/home/antonis/datasets/huggingface/wmt09_gens")

In [ ]:
wmt09['en-hu']['translation']

In [ ]:
wmt09['en-hu']['translation'][0]['gen'] = 1
wmt_dict = wmt09['en-hu'].to_dict()
wmt_dict['translation'][0]['gen'] = 1

In [ ]:
wmt_dict['translation'][0]

In [ ]:
gens = [1] * len(wmt09['en-hu']['translation'])
wmt09['en-hu']['translation']['gen'] = gens

In [ ]:
type(wmt09)

In [ ]:
# wmt_09 = _load_dataset("wmt")
trivia_qa_ngram_pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/results/n-grams/trivia_qa/pile/exp_3/test-set/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaFalse/5/all/trivia_qa.pkl"
trivia_qa = pd.DataFrame(pd.read_pickle(trivia_qa_ngram_pth)).T.sort_values('value', ascending=False)

In [ ]:
trivia_qa

In [ ]:
model_instance_pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/2/model_instance_results_0-shot_common.pkl"
with open(model_instance_pth, "rb") as f:
    model_instance = pickle.load(f)

In [ ]:
wmt_res_pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/2/examples_dfs_0-shot_common_models.pkl"
with open(wmt_res_pth, "rb") as f:
    wmt_res = pickle.load(f)

In [ ]:
wmt_res

In [ ]:
trivia_qa = load_dataset("trivia_qa", "unfiltered.nocontext", cache_dir=CACHE_DIR)

In [ ]:
trivia_qa['test'][1].keys()

In [ ]:
trivia_qa['validation'][1]['answer']

In [ ]:
trivia_qa['test'][1]

# print 10 question answer pairs from test set

for i in range(10):
    print(trivia_qa['test'][i]['question'])
    print(trivia_qa['test'][i]['answer'])
    print("\n")

In [ ]:
trivia_qa['test'][1]['question']

In [ ]:
trivia_qa['test'][1]['search_results'].keys()

In [ ]:
# # bigbench_tasks = ['bigbench_causal_judgement', 'bigbench_date_understanding', 'bigbench_disambiguation_qa', 'bigbench_dyck_languages', 'bigbench_formal_fallacies_syllogisms_negation', 'bigbench_geometric_shapes', 'bigbench_hyperbaton', 'bigbench_logical_deduction_five_objects', 'bigbench_logical_deduction_seven_objects', 'bigbench_logical_deduction_three_objects', 'bigbench_movie_recommendation', 'bigbench_navigate', 'bigbench_reasoning_about_colored_objects', 'bigbench_ruin_names', 'bigbench_salient_translation_error_detection', 'bigbench_snarks', 'bigbench_sports_understanding', 'bigbench_temporal_sequences', 'bigbench_tracking_shuffled_objects_five_objects', 'bigbench_tracking_shuffled_objects_seven_objects', 'bigbench_tracking_shuffled_objects_three_objects', 'blimp_adjunct_island']
# # save bigbench tasks as txt
# path = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/configs"
# with open(os.path.join(path, "bigbench_tasks.txt"), "w") as f:
#     for task in bigbench_tasks:
#         f.write(task + "\n")

In [ ]:
configs = get_dataset_config_names("hendrycks_test")

ds_full = {}
for config in configs:
    ds_full[config] = load_dataset("hendrycks_test", config, cache_dir=CACHE_DIR)
    break

In [ ]:
ds_full['abstract_algebra']['test']['choices'][0]

In [ ]:
# load bigbench tasks
bigbench_tasks = []
with open("./configs/data/bigbench_tasks.txt", "r") as f:
    for line in f:
        bigbench_tasks.append(line.strip())
bigbench_tasks

In [ ]:
configs = get_dataset_config_names("bigbench")
configs

ds_full = {}
for config in configs:
    if config in bigbench_tasks:
        ds_full[config] = load_dataset("bigbench", config, 
                                       cache_dir=CACHE_DIR,
                                       trust_remote_code=True)

In [ ]:
ds_logical = load_dataset("bigbench", "logical_deduction")

In [ ]:
for name, ds in ds_full.items():
    print(name, ds)

In [ ]:
ds_1 = ds_full['abstract_algebra']

In [ ]:
ds_1_concat = concatenate_datasets([ds for ds in ds_1.values()])
ds_1_concat

In [ ]:
ds_full['abstract_algebra']['auxiliary_train'][0]

In [ ]:
ds_full.keys()

In [ ]:
ds_full['abstract_algebra']['test'][0:2]

In [ ]:
from nltk import ngrams

example = ds[15]['answer']
print(str(example))

text_gram = ngrams("hello", 5)

print(list(text_gram))

text_gram = [''.join(ngram) for ngram in text_gram]

print(text_gram)

In [ ]:
for key in ds_full.keys():
    ds = ds_full[key]['test']
    print(key, ds)
    print(ds[10])

In [ ]:
lang_pair = "es-en"
dataset_name = "wmt-14"

datasets = load_dataset("tasksource/mmlu", cache_dir=CACHE_DIR)

In [ ]:
# ds = load_dataset("SirNeural/flan_v2", cache_dir=CACHE_DIR)
ds_task = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds")

In [ ]:
ds_task

In [ ]:
corpus = "EleutherAI/the_pile_deduplicated"
corpus_path = f"/share/edc/home/antonis/datasets/huggingface/flan_v1/ds_{corpus}_small"
# ds = load_from_disk(corpus_path)

In [ ]:
print(corpus_path)

In [ ]:
ds_task['NLI']

In [ ]:
import random

task_keys = list(ds_task.keys())
n_index = random.randint(0, len(task_keys))
task = ds_task[task_keys[n_index]]
n_index_2 = random.randint(0, len(task))

print(task[n_index_2]["text"])


In [ ]:
task_examples = collections.defaultdict(lambda: collections.defaultdict(dict))
for task in task_ds.keys():
    task_dataset = task_ds[task]
    # sample 10 examples
    task_dataset = task_dataset.shuffle(seed=42).select(range(20))
    for n, example in enumerate(task_dataset):
        task_examples[task][n]['inputs'] = example['inputs']
        task_examples[task][n]['targets'] = example['targets']

# save task_examples as jsonl
with open("/share/edc/home/antonis/datasets/huggingface/flan_v1_task_examples.jsonl", "w") as f:
    for task in task_examples.keys():
        for n, example in task_examples[task].items():
            # Create a new dictionary for each line, with 'task', 'example_number' and 'text' keys
            line = {'task': task, 'example_number': n, 'text': example}
            f.write(json.dumps(line, indent=4) + "\n")

In [ ]:
#load 20% of c4
c4 = load_dataset("c4", "en", split="train[:20%]")

In [ ]:
import collections
import pickle

max_len = 1000
n_groups = 10
model_name_or_path = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/pythia/experiment_1/huggingface/flan_v1/c4_mixed_Commonsense/EleutherAI/pythia-1.4B-deduped_ckpt_False/checkpoint-70000"
task_samples_path = f"/share/edc/home/antonis/datasets/huggingface/flan_v1/task_ds_sampled_{max_len}_{n_groups}.pkl"
dataset_dict_path = f"/share/edc/home/antonis/datasets/huggingface/flan_v1/ds_c4_small_sampled_{max_len}_{n_groups}"
device = "cpu"


with open(task_samples_path, 'rb') as f:
    task_samples_dict = collections.defaultdict(dict, pickle.load(f))

ds = load_from_disk(dataset_dict_path)

grad_pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/grads_c4.pkl"


In [ ]:
with open(grad_pth, 'rb') as f:
    grads = pickle.load(f)

grads.keys()

In [ ]:
len(grads['0'][0])

In [ ]:
import torch
gradient_mags = gradient_mags = [torch.norm(grad[0]).item() for grad in grads['0']]

In [ ]:
import os
import shutil

# Specify the directory containing your dataset files
dataset_dir = "/share/edc/home/antonis/datasets/huggingface/flan_v1/c4_mixed_QA_NLI_Summarization_Commonsense"

# Create subdirectories for each cache type
cache_types = ["0e1ee472afc16a84", "93decdfdc22cb71c", "a516fcb03f1e5f67", "cc79b545cdc85cd7"]
for cache_type in cache_types:
    os.makedirs(os.path.join(dataset_dir, cache_type), exist_ok=True)

# Move each file to the appropriate subdirectory
for filename in os.listdir(dataset_dir):
    for cache_type in cache_types:
        if filename.startswith(f"cache-{cache_type}"):
            shutil.move(os.path.join(dataset_dir, filename), os.path.join(dataset_dir, cache_type, filename))

In [ ]:
import os
import shutil

# Specify the directory containing your dataset files
dataset_dir = "/share/edc/home/antonis/datasets/huggingface/flan_v1/c4_mixed_QA_NLI_Summarization_Commonsense"

# Specify the cache types
cache_types = ["0e1ee472afc16a84", "93decdfdc22cb71c", "a516fcb03f1e5f67", "cc79b545cdc85cd7"]

# Move each file back to the original directory
for cache_type in cache_types:
    cache_dir = os.path.join(dataset_dir, cache_type)
    for filename in os.listdir(cache_dir):
        shutil.move(os.path.join(cache_dir, filename), os.path.join(dataset_dir, filename))

In [ ]:
cache_path = os.path.join(dataset_dir, cache_types[0])

cache_ds = load_from_disk(cache_path)

In [ ]:
def print_example(example):
    for key in example.keys():
        print(f"{key}: {example[key]}")

In [ ]:
# ds = load_dataset("conceptofmind/dialog_submix_original", split="train")
# ds = load_dataset("conceptofmind/cot_submix_original", split="train")
ds = load_dataset("conceptofmind/flan2021_submix_original", split="train", cache_dir=CACHE_DIR)
# ds = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1/c4_mixed_NLI")

In [ ]:
# ds_task = load_from_disk('/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds')
ds_c4_small = load_from_disk("/share/edc/home/antonis/datasets/huggingface/flan_v1/ds_c4_small")

In [ ]:
filtered_ds = ds_task['NLI'].filter(lambda example: 'anli/r2' in example['task_name'])

In [ ]:
filtered_ds['inputs'][0:10]

In [ ]:
import random

sentiment_ds = ds_task['Sentiment']

n_samples = 20
n_iter = 0
while n_iter < n_samples:
    index = random.randint(0, len(sentiment_ds) - 1)
    row = sentiment_ds[index]
    if row['task_name'] == 'glue/sst2:2.0.0':
        print(f"------ inputs: {row['inputs']}")
        print(f"targets: {row['targets']}")
        n_iter += 1

In [ ]:
ds_t0 = load_dataset("conceptofmind/t0_submix_original", cache_dir=CACHE_DIR, split="train")

In [ ]:
# ds_t0 = concatenate_columns(ds_t0, 'inputs', 'targets', 'text')
# count_gpt2_tokens(ds_t0, 'text')

In [ ]:
ds_cot = load_dataset("conceptofmind/cot_submix_original", cache_dir=CACHE_DIR, split="train")
ds_cot = concatenate_columns(ds_cot, 'inputs', 'targets', 'text')
n_cot_tokens = count_gpt2_tokens(ds_cot, 'text')
print(f"{n_cot_tokens:.2e}")

In [ ]:
print(f"{n_cot_tokens:.2e}")

In [ ]:
targets_dict = {}
for key in ds_task:
    unique_targets = ds_task[key].unique('targets')
    short_targets = [target for target in unique_targets if len(target) < 20]
    targets_dict[key] = short_targets

In [ ]:
import json
with open('/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds/short_targets.json', 'w') as f:
    json.dump(targets_dict, f, indent=4)

In [ ]:
# load file
with open('/share/edc/home/antonis/datasets/huggingface/flan_v1_task_ds/short_targets.json', 'r') as f:
    targets_dict = json.load(f)

In [ ]:
# is str in list

strings = ["it is not possible to tell", "pick from the following"]
task = "NLI"
for string in strings:
    if string in targets_dict[task]:
        print(f"{string} in {task}")
    else:
        print(f"{string} not in {task}")

In [ ]:
ds_flan = load_dataset("declare-lab/flan-mini", subset="Flan2021")

In [ ]:
import re

def extract_options(input_string):
    # Define a regular expression pattern for the options phrase
    pattern = r"(options are|OPTIONS|options):?\s*(.*)"
    # Search for the pattern in the input string, ignoring case
    match = re.search(pattern, input_string, re.IGNORECASE)
    if match:
        # If a match is found, split the matched string on '--' to get individual options
        options = match.group(2).split('--')
        # Remove leading and trailing whitespace from each option
        options = [option.strip() for option in options if option]
        return options
    else:
        return []

print(extract_options(ds_task['NLI']['inputs'][1]))

In [ ]:
len(set(ds['task_name']))
set(ds['task_name'])

In [ ]:
required_tasks = [
    "anli",
    "rte",
    "cb",
    "snli",
    "mnli",
    "wnli",
    "qnli"
]

ds1 = ds.filter(lambda example: any(task in example['task_name'] for task in required_tasks))




In [ ]:
def concatenate_columns(dataset, column1, column2, new_column_name):
    def concat_example(example):
        example[new_column_name] = example[column1] + " " + example[column2]
        return example

    return dataset.map(concat_example)

# Usage example:
ds1 = concatenate_columns(ds1, 'inputs', 'targets', 'text')

In [ ]:
from transformers import GPT2Tokenizer
from tqdm import tqdm

def total_tokens(dataset, text_field):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    total_count = 0
    
    for example in tqdm(dataset, desc="Counting tokens"):
        text = example[text_field]
        tokens = tokenizer.tokenize(text)
        total_count += len(tokens)

    # Print the total count in scientific notation
    print(f"Total number of tokens: {total_count:.2e}")

    return total_count

# Usage example:
total_count = total_tokens(ds1, 'text')

In [ ]:
print(f"Total number of tokens: {total_count:.2e}")

In [ ]:
# ds['label']

# # get unique labels
# labels = set(ds['label'])
# print(labels)

In [ ]:
ds = load_from_disk("/share/edc/home/antonis/datasets/huggingface/merged_datasets/NLI/P_1_PQA_5_promptsource_False/dataset_1/dataset_train.arrow")

In [ ]:
labels = ds['train']['label']

# find -1 in labels
non_matching = []
for i in range(len(labels)):
    if labels[i] == -1:
        print(i)
        non_matching.append(i)
        print_example(ds['train'][i])

In [ ]:
# search revursively for .arrow file
import glob
import json

base_dir = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_unified_labels/P_1_PQA_5_promptsource_False"

arrow_files = glob.glob(base_dir + "/**/*.arrow", recursive=False)
print(arrow_files)

for file_ in arrow_files:
    print(file_)
    ds = load_from_disk(file_)
    print(ds)

json_files = glob.glob(base_dir + "/**/*.json", recursive=False)
print(json_files)


# def print_keys_values(data):
#     for key, value in data.items():
#         print(key)
#         if isinstance(value, dict):
#             print_keys_values(value)
#         else:
#             print(value)

for file_ in json_files:
    print(file_)
    with open(file_, 'r') as f:
        data = json.load(f)
    # print keys and values recursively
    # print_keys_values(data)
    print(data)



In [ ]:
arrow_files

In [ ]:
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_train.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/dataset_1/dataset_validation.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/P_QA_5/dataset_1/dataset_validation.arrow"
# dataset_c4 = load_dataset("c4", "en", split="train[:20%]", cache_dir=CACHE_DIR)

ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_train.arrow"
dataset = load_from_disk(ds_pth)

# ds_path2 = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_train.arrow"
# dataset2 = load_from_disk(ds_path2)

ds_pth_validation = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_validation.arrow"
dataset_validation = load_from_disk(ds_pth_validation)

In [ ]:
len(ds)

In [ ]:
ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
ds_sst = load_dataset("sst", split="test", cache_dir=CACHE_DIR)

print(f"IMDB len: {len(ds_imdb)}")
print(f"SST len: {len(ds_sst)}")

In [ ]:
# first protion is IMDB, second is SST
ds_pth_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_validation.arrow"
dataset_val = load_from_disk(ds_pth_val)
ds_pth2_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_validation.arrow"
dataset2_val = load_from_disk(ds_pth2_val)

In [ ]:
import json

ds_path = "/share/edc/home/antonis/datasets/huggingface/C4/limit_total_tokens_1240000000/train"
ds = load_from_disk(ds_path)

In [ ]:
print(f"train1: {len(dataset)}")
print(f"train2: {len(dataset2)}")
print(f"val1: {len(dataset_val)}")
print(f"val2: {len(dataset2_val)}")

In [ ]:
dataset_val.select(range(len(dataset2_val)))

In [ ]:
# dataset_val[-len(dataset2_val):]

dataset_val1 = dataset_val.select(range(len(ds_imdb)))
dataset_val2 = dataset_val.select(range(len(ds_imdb), len(dataset_val)))

assert len(dataset_val1) + len(dataset_val2) == len(dataset_val), f"len mismatch: {len(dataset_val1)} + {len(dataset_val2)} != {len(dataset_val)}"

def add_new_column1(example, keyword, column_name="dataset"):
    example[column_name] = keyword
    return example

dataset_val1 = dataset_val1.map(lambda x: add_new_column1(x, "imdb"))
dataset_val2 = dataset_val2.map(lambda x: add_new_column1(x, "sst"))

dataset_val = concatenate_datasets([dataset_val1, dataset_val2])

# save the new dataset
dataset_val.save_to_disk(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))

In [ ]:
print(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))

In [ ]:
import os
import json
import logging
from datasets import load_dataset, Dataset
from promptsource.templates import DatasetTemplates, TemplateCollection
# set logging level to INFO
from src._promptsource import get_T0MixtureDatasets 
logger = logging.getLogger(__name__)
logger.setLevel(20)

In [ ]:
TOMixture = [
    ("sst","default"), # Senitment Classification")
    ("imdb",None),
]

In [ ]:
# datasets = get_T0MixtureDatasets("test", max_samples=1000, return_as_dict=True)
# ds_sst = load_dataset("sst", "default", split="test", cache_dir=CACHE_DIR)
ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
# ds_yelp_review = load_dataset("yelp_review_full", split="train", cache_dir=CACHE_DIR)
ds_sentiment140 = load_dataset("sentiment140", split="test", cache_dir=CACHE_DIR)

print(f"IMDB len: {len(ds_imdb)}")
# print(f"Yelp len: {len(ds_yelp_review)}")
print(f"Sentiment140 len: {len(ds_sentiment140)}")

In [ ]:
name, subset = TOMixture[0]
example = ds_sst.select(range(1))
# DatasetTemplates(name, subset).templates.get_get_answer_choices_list(example)

In [ ]:
print(len(ds_sst))
print(len(ds_imdb))
# print(len(ds_yelp_review))
# print(len(ds_sentiment140))

import random

ds_name = datasets['imdb']

def apply_template(example, template, dataset_name):
    if isinstance(template, list):
        template = random.choice(template)
    try:
        answer_choices = template.get_answer_choices_list(example)
    except:
        answer_choices = None
    # example['label'] = float(example['label'])
    return {"prompt": template.apply(example), "answer_choices": answer_choices, "dataset": dataset_name}

# # You can apply the template to the dataset
# prompted_dataset = ds_name.map(lambda example: apply_template(example, ds_name.templates))

def create_prompted_dataset(datasets):
    prompted_datasets = {}
    for name, dataset in datasets.items():
        prompted_dataset = dataset.map(lambda example: apply_template(example, dataset.templates, name))
        # columns to keep
        columns_to_keep = ["prompt", "answer_choices", "dataset"]
        prompted_dataset = prompted_dataset.remove_columns([col for col in prompted_dataset.column_names if col not in columns_to_keep])
        prompted_datasets[name] = prompted_dataset

    prompted_datasets = concatenate_datasets([dataset for name, dataset in prompted_datasets.items()])
    return prompted_datasets

prompted_datasets = create_prompted_dataset(datasets)

In [ ]:
# save prompted dataset

# prompted_datasets.save_to_disk("/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_test_with_answers.arrow")

In [ ]:
for name, dataset in datasets.items():
    print(name, dataset)
    print(dataset.templates)